# Main ...

 
``config, load, preprocess, train, eval  for  Tick tick bloom``

**Yo DON'T rerun this unless you want to overwrite past models, always fork and do your stuff and 
DON'T forget to change the name**

**``Mission: Model nns only on metadata!``**

# Load imports and dependencies

In [81]:
import warnings
import sys
import os
import time
import joblib
import random
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold

import tensorflow as tf
from tensorflow.keras import layers, activations, losses, metrics, models, optimizers, callbacks
from category_encoders.target_encoder import TargetEncoder

warnings.filterwarnings('ignore')

In [40]:
# local utilities imports
from tick_tick_bloom_utils import my_keras_rmse, comp_metric, den2sev_map

In [41]:
# wandb stuff for tracking
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_login = user_secrets.get_secret("wandb_bloom_tracker")

import wandb
wandb.login(key=wandb_login)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Config

In [82]:
# dot dictionary
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


# Config
config = {}
config = dotdict(config)
config['RANDOM_SEED'] = 18952


config['unique_id'] = int(time.time())
print(f'unique_id: {config.unique_id}')
config['name'] = f'd128_rmse_lndsat8_raw_v1-{config.unique_id}'

config['PROJECT_NAME'] = 'tick-tick-bloom'
# config['DATA_DIR'] = '../data/'
# config['MODEL_DIR'] = '../models/'
config['SAVE_MODEL'] = True


# # Img config
# config['IMG_SIZE'] = (136, 136)
# config['CHANNELS'] = 3

# training configuration
config['train'] =  dotdict({
                        'epochs': 50,
                        'batch_size': 128,
                        'validation_split': 0.2,
                        'shuffle': True,
                        'verbose': 1
                        })

config['desc'] = """ xp2-to see how nns perform only on metadata with rmse loss""" 

unique_id: 1673284256


In [43]:
# seed everything
random.seed(config.RANDOM_SEED)
np.random.seed(config.RANDOM_SEED)
tf.random.set_seed(config.RANDOM_SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['PYTHONHASHSEED'] = str(config.RANDOM_SEED)

# tf.keras.utils.set_random_seed(config.RANDOM_SEED)  # supposedly sets seed for python, numpy, tf

# Load data

In [112]:
INPUT_METADATA_DIR = '/kaggle/input/ticktickbloomdataset'

metadata = pd.read_csv(os.path.join(INPUT_METADATA_DIR, 'metadata.csv'))
sub_format = pd.read_csv(os.path.join(INPUT_METADATA_DIR, 'submission_format.csv'))
train_labels = pd.read_csv(os.path.join(INPUT_METADATA_DIR, 'train_labels.csv'))

In [45]:
# IMG_DIR = '/kaggle/input/pull-landsat-data-v1-500m/landsat8_500m_v1'   # landsat 8 data with raw 
# img_files = os.listdir(IMG_DIR)
# img_file_names = [f.split('.')[0] for f in img_files]

# # get only data for those 1k imgs
# metadata_subset = metadata[metadata['uid'].isin(img_file_names)]
# data = metadata_subset[metadata_subset.split == 'train']
# data = data.merge(train_labels, on='uid')

# test_data = metadata[metadata.split == 'train']

In [46]:
# def get_imgs(uids) :
#     imgs = []
#     for uid in uids:
#         img_arr = np.load(IMG_DIR + f'/{uid}.npy')
#         img_arr = np.transpose(img_arr, (2, 1, 0))
#         # resize img
#         img_arr = cv2.resize(img_arr, config.IMG_SIZE)
#         img_arr = img_arr / 255   # normalizeee bro... other wise it's blowing up the networks...
#         imgs.append(img_arr)
#     return np.array(imgs) 


# def get_np_data(split : float = 0.2, task='train'):
#     """Return np data for training and validation."""
#     if task == 'train':
#         print("Loading train and validation data...")
#         x_train_uids, x_val_uids, y_train, y_val = train_val_split(
#             data['uid'],
#             data.severity,
#             val_size=split,
#             random_state=config.RANDOM_SEED,
#             stratify=data.severity
#         )

#         x_train = get_imgs(x_train_uids)
#         x_val = get_imgs(x_val_uids)

#         return x_train, y_train, x_val, y_val


#     if task == 'test':
#         test_ids = test_data.uids
#         x_test
#         return x_test

In [47]:
# x_train, y_train, x_val, y_val = get_np_data()
# print(y_train.value_counts(normalize=True))
# print(y_val.value_counts(normalize=True))
# print('Done')

In [118]:
#  get train data
metadata.date = pd.to_datetime(metadata.date)
data = pd.merge(metadata, train_labels, on='uid', how='left')
print(data.shape)


# seasons
seasons = {
    1: 1,
    2: 1,
    3: 2,
    4: 2,
    5: 2,
    6: 3,
    7: 3,
    8: 3,
    9: 4,
    10: 4,
    11: 4,
    12: 1
}

# add date time fts.
data['month'] = data.date.dt.month
data['year'] = data.date.dt.year
data['season'] = data.month.map(seasons)


all_train = data[data.split == 'train']
all_train.shape

(23570, 8)


(17060, 11)

In [124]:
test_data = metadata[metadata.split == 'test']
test_data = test_data.merge(sub_format, on='uid')

test_data['month'] = test_data.date.dt.month
test_data['year'] = test_data.date.dt.year
test_data['season'] = test_data.month.map(seasons)

In [49]:
train, val = train_test_split(all_train, test_size=0.20, random_state=config.RANDOM_SEED, stratify=all_train.severity)
train.shape, val.shape

((13648, 11), (3412, 11))

In [50]:
X_train_, y_train_ = train[['region', 'month', 'year', 'season']], train['severity'] 
X_val_, y_val_ = val[['region', 'month', 'year', 'season']], val['severity']

X_train_.shape, y_train_.shape, X_val_.shape, y_val_.shape

((13648, 4), (13648,), (3412, 4), (3412,))

In [127]:
X_test_ = test_data[['region', 'month', 'year', 'season']]
X_test_.shape

(6510, 4)

# Preprocess

In [51]:
# # change labels to 0-3(model works this way) instead of 1-4 given range(given severity)
# -1 for to make labels look like sparse encoded labels

y_train = y_train_  -1
y_val = y_val_ - 1


y_train.value_counts(normalize=True), y_val.value_counts(normalize=True)  # guessing alwyas 0 gives 43% acc

(0    0.439478
 3    0.207943
 1    0.189845
 2    0.159364
 4    0.003370
 Name: severity, dtype: float64,
 0    0.439332
 3    0.207796
 1    0.189918
 2    0.159437
 4    0.003517
 Name: severity, dtype: float64)

In [52]:
#  target encode the cat fts.

te = TargetEncoder(cols=['region', 'month', 'year', 'season'])
te.fit(X_train_, y_train)
X_train =  te.transform(X_train_)
X_val = te.transform(X_val_)

In [128]:
X_test = te.transform(X_test_)
X_test

,region,month,year,season
0,2.752492,1.074234,1.020629,1.080931
1,2.752492,1.163842,1.311203,1.144245
2,0.794212,1.074234,1.085352,1.080931
3,1.199777,1.074234,1.206065,1.080931
4,2.752492,1.627866,1.059012,1.417059
...,...,...,...,...
6505,0.794212,1.187065,1.206065,1.080931
6506,2.752492,1.164905,1.206065,1.144245
6507,0.564009,1.187065,1.206065,1.080931
6508,2.752492,1.108734,1.311203,1.417059


In [53]:
display(X_train.head())
display(X_val.head())

,region,month,year,season
9569,0.564009,0.927402,1.020629,1.129643
1678,0.564009,1.164905,1.311203,1.144245
11833,0.794212,1.164905,1.311203,1.144245
12637,0.564009,0.981379,1.126844,1.080931
2840,2.752492,0.981379,1.305835,1.080931


,region,month,year,season
14227,0.564009,0.981379,0.904943,1.080931
10183,0.564009,1.187065,1.311203,1.080931
6578,2.752492,1.510490,1.311203,1.417059
5824,2.752492,0.981379,1.305835,1.080931
551,1.199777,1.187065,1.305835,1.080931


# Model

In [68]:
def comp_loss(y_true, y_pred):
    return tf.sqrt(losses.mean_squared_error(y_true, y_pred))

In [69]:
def get_model():
    print('Loading model...')
#     input_shape = (*config.IMG_SIZE, config.CHANNELS)

    input_imgs = layers.Input(shape=(X_train.shape[1],))
#     x = layers.Conv2D(32, (3, 3), activation='relu')(input_imgs)
#     x = layers.MaxPooling2D((2, 2))(x)
#     x = layers.Conv2D(32, (3, 3), activation='relu')(input_imgs)
#     x = layers.MaxPooling2D((2, 2))(x)
#     x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(input_imgs)
    output = layers.Dense(1)(x)     # regressing the severity level !

    model = models.Model(inputs=input_imgs, outputs=output, name=config.name)

    model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
                    loss=comp_loss,
                    metrics=[
                        comp_loss,
                        metrics.SparseCategoricalAccuracy(name='acc')
                    ])

    return model

In [70]:
model = get_model()
model.summary()

Loading model...
Model: "d128_rmse_lndsat8_raw_v1-1673283452"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)               640       
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 129       
Total params: 769
Trainable params: 769
Non-trainable params: 0
_________________________________________________________________


# Train and eval

In [71]:
# np and tf metrics are aggreable, but why evaluate and final epoch's loss and metric each, are different?!
# 1.5447352189134953 (np) 1.5447352189134953 (tf) , 1.5175646543502808(evaluate), 


# >> that's coz final epoch also has some batchs which are worst performing and model is still learning and final epoch loss is avg of all those batches.
# whereas evaluete the model's fixed and no leariing believe in evaluate!

In [76]:
def train(model=get_model(), config=config, x_train=X_train, y_train=y_train):
    """fits given model to x_train and y_train"""
    
    train_config = config['train']
    my_callbacks = []
    
    earlystopping = callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    my_callbacks.append(earlystopping)
    
    try:
        wandb_callback = wandb.keras.WandbCallback(
            monitor='val_loss',
            log_weights=True,
            log_gradients=True,
            save_model=False,
            training_data=(x_train, y_train),
            log_batch_frequency=None,
        )

        my_callbacks.append(wandb_callback)
    except:
        print('wandb not tracking')
        
    print(f'Training model... {config.name}')
    history = model.fit(
                x_train, y_train,
                epochs=train_config.epochs,
                batch_size=train_config.batch_size, 
                callbacks=my_callbacks, 
                validation_split=0.2, 
                shuffle=True, 
                verbose=1   
            )

    return model, history

Loading model...


In [77]:
def eval_model(model, x_val=X_val, y_val=y_val):
    print('Evaluating model....')
    model.evaluate(x_val, y_val, return_dict=True)


In [83]:
with wandb.init(project=config.PROJECT_NAME, config=config, name=config.name):
    
    # load nwe model or old saved model
    model = get_model()

    # train
    model, history = train(model, config, y_train=y_train)
    # eval
    eval_model(model)
    
    # classification report
    y_pred = model.predict(X_val)
    y_pred_hard = np.round(y_pred)
    error = mse(y_val, y_pred_hard, squared=False)
    print("Comp Metric: ", error)
    cr = classification_report(y_val+1, y_pred_hard+1)     # +1 to account for 0-4 as it should be 1-5 originallly
    print(cr)
    

Loading model...
Training model... d128_rmse_lndsat8_raw_v1-1673284256
Epoch 1/50


2023-01-09 17:11:27.086418: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2023-01-09 17:11:27.086729: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-01-09 17:11:27.088191: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.01ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.



86/86 [==============================] - 1s 6ms/step - loss: 1.1189 - comp_loss: 1.1189 - acc: 0.4392 - val_loss: 1.0777 - val_comp_loss: 1.0777 - val_acc: 0.4407
Epoch 2/50
86/86 [==============================] - 0s 4ms/step - loss: 1.0257 - comp_loss: 1.0257 - acc: 0.4392 - val_loss: 0.9811 - val_comp_loss: 0.9811 - val_acc: 0.4407
Epoch 3/50
86/86 [==============================] - 0s 3ms/step - loss: 0.9331 - comp_loss: 0.9331 - acc: 0.4392 - val_loss: 0.8918 - val_comp_loss: 0.8918 - val_acc: 0.4407
Epoch 4/50
86/86 [==============================] - 0s 3ms/step - loss: 0.8477 - comp_loss: 0.8477 - acc: 0.4392 - val_loss: 0.8138 - val_comp_loss: 0.8138 - val_acc: 0.4407
Epoch 5/50
86/86 [==============================] - 0s 3ms/step - loss: 0.7878 - comp_loss: 0.7878 - acc: 0.4392 - val_loss: 0.7651 - val_comp_loss: 0.7651 - val_acc: 0.4407
Epoch 6/50
86/86 [==============================] - 0s 4ms/step - loss: 0.7445 - comp_loss: 0.7445 - acc: 0.4392 - val_loss: 0.7202 - val_com

acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
comp_loss,█▇▆▅▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
loss,█▇▆▅▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_comp_loss,█▇▆▄▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▆▄▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
GFLOPS,0.0
acc,0.43918
best_epoch,45
best_val_loss,0.55192


ValueError: autodetected range of [nan, nan] is not finite

In [79]:
# history.history

In [93]:
model = models.load_model('/kaggle/working/d128_rmse_lndsat8_raw_v1-1673283452.h5', custom_objects={'comp_loss': comp_loss})
preds = model.predict(X_val)
int_preds = np.round(preds)
mse(y_val, int_preds, squared=False)

0.9777717088636599

In [133]:
int_preds.shape

(3412, 1)

In [134]:
pd.Series(int_preds.ravel()).value_counts()  # no 3 and 5

0.0    2245
3.0     759
1.0     408
dtype: int64

# Save anything..

In [84]:
# save model
if config.SAVE_MODEL:
    model.save(config.name + '.h5')
    print("Model saved as ",config.name + '.h5')

Model saved as  d128_rmse_lndsat8_raw_v1-1673284256.h5


# Make submission

In [95]:
model.summary()

Model: "d128_rmse_lndsat8_raw_v1-1673283452"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
dense_32 (Dense)             (None, 128)               640       
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 129       
Total params: 769
Trainable params: 769
Non-trainable params: 0
_________________________________________________________________


In [140]:
test_preds = np.round(model.predict(X_test)).ravel()
test_preds = test_preds + 1
sub_format.severity = test_preds
sub_format.severity = sub_format.severity.astype(int) 
sub_format.severity.value_counts()

1    2558
4    2387
2    1565
Name: severity, dtype: int64

In [142]:
sub_format.to_csv(f'{config.name}_preds.csv', index=False) # expecting @ 0.9777